<a href="https://colab.research.google.com/github/VincentHenric/Deep_learning_project/blob/master/DL_Proj_Corentin.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
  
import tensorflow as tf

TensorFlow 2.x selected.


In [3]:
try:
  import tensorflow.compat.v2 as tf
except Exception:
  pass

tf.enable_v2_behavior()

print(tf.__version__)

2.0.0


In [4]:
#if pybind11 is not installed
pip install pybind11

     |████████████████████████████████| 153kB 2.8MB/s 


In [5]:
#Connect to our own Google Drive
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [6]:
#Go on the DL project folder
%cd "/content/gdrive/My Drive/DeepLearningProject/"
!ls

/content/gdrive/My Drive/DeepLearningProject
Board.h			games.data				models
compileMAC.sh		games.txt				policy.npy
compile.sh		golois.cpp				__pycache__
convert			golois.cpython-36m-x86_64-linux-gnu.so	README
convert.cpp		golois.cpython-37m-x86_64-linux-gnu.so	Rzone.h
DL_Proj_Corentin.ipynb	golois.py				save.py
end.npy			input_data.npy				value.npy
Game.h			ls.sh


In [6]:
#To compile golois.cpp
#! c++ -O3 -Wall -shared -std=c++11 -fPIC `python3 -m pybind11 --includes` golois.cpp -o golois`python3-config --extension-suffix`

In file included from golois.cpp:17:0:
Board.h: In member function ‘bool Board::isCapturedLadder(int, int, Rzone*)’:
Board.h:1736:8: warning: unused variable ‘n1’ [-Wunused-variable]
    int n1 = nbLiberties (inter, liberties1, stones1, 3);
        ^~
Board.h:1757:12: warning: unused variable ‘n1’ [-Wunused-variable]
        int n1 = nbLiberties (inter, liberties1, stones1, 3);
            ^~
Board.h: In member function ‘void Board::computeLadders(int)’:
Board.h:1786:9: warning: unused variable ‘other’ [-Wunused-variable]
     int other = opponent (color);
         ^~~~~
Board.h: In member function ‘void Board::computeAllLadders(int, bool)’:
Board.h:2032:9: warning: unused variable ‘n1’ [-Wunused-variable]
     int n1 = nbLiberties (i, liberties1, stones1);
         ^~
Board.h:2068:7: warning: unused variable ‘n1’ [-Wunused-variable]
   int n1 = nbLiberties (i, liberties1, stones1);
       ^~
Board.h:2101:9: warning: unused variable ‘n1’ [-Wunused-variable]
     int n1 = nbLiberties (i

In [9]:
import tensorflow as tf
import tensorflow.keras as keras
import numpy as np
from tensorflow.keras import layers 
from datetime import datetime
import os
import golois

from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

from tensorflow.keras.layers import Dense, Flatten, Input, Dropout, MaxPooling2D, Conv2D, BatchNormalization, Activation, AveragePooling2D, ZeroPadding2D
from tensorflow.keras import optimizers
from tensorflow.keras import regularizers
from tensorflow.keras.regularizers import l2


planes = 8
moves = 361
dynamicBatch = True
if dynamicBatch:
    N = 300000
    input_data = np.random.randint(2, size=(N, 19, 19, planes))
    input_data = input_data.astype ('float32')
    
    policy = np.random.randint(moves, size=(N,))
    policy = keras.utils.to_categorical (policy)
    
    value = np.random.randint(2, size=(N,))
    value = value.astype ('float32')
    
    end = np.random.randint(2, size=(N, 19, 19, 2))
    end = end.astype ('float32')

    golois.getBatch (input_data, policy, value, end)
else:
    input_data = np.load ('input_data.npy')
    policy = np.load ('policy.npy')
    value = np.load ('value.npy')
    end = np.load ('end.npy')

input = keras.Input(shape=(19, 19, planes), name='board')

x = layers.Conv2D(42, 3, padding='same')(input)
x = BatchNormalization()(x)
x = Activation('relu')(x)

x = layers.Conv2D(128, 3, padding='same')(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = Dropout(0.3)(x)

ident = x
x = layers.Conv2D(128, 3, padding='same')(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = layers.Conv2D(128, 3, padding='same')(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = Dropout(0.3)(x)
x = layers.add([ident,x])
ident2 = x
x = layers.Conv2D(128, 3, padding='same')(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = layers.Conv2D(128, 3, padding='same')(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = Dropout(0.3)(x)
x = layers.add([ident2,x])

policy_head = layers.Conv2D(2, 1, padding='same')(x)
policy_head = BatchNormalization()(policy_head)
policy_head = Activation('relu')(policy_head)
policy_head = layers.Flatten()(policy_head)
policy_head = layers.Dense(moves, activation='softmax', name='policy')(policy_head)

value_head = layers.Conv2D(1, 1, padding='same')(x)
value_head = BatchNormalization()(value_head)
value_head = Activation('relu')(value_head)
value_head = layers.Flatten()(value_head)
value_head = layers.Dense(256, activation='relu')(value_head)
value_head = layers.Dense(1, activation='tanh', name='value')(value_head)

model = keras.Model(inputs=input, outputs=[policy_head, value_head])
model.summary()

#Name of the model, to save it later
now = datetime.now()
date_time = now.strftime("%m-%d-%Y_%H-%M-%S")
save_dir = os.path.join(os.getcwd(), 'models')
model_name = 'dl_proj_model_' + date_time + '.h5'



Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
board (InputLayer)              [(None, 19, 19, 8)]  0                                            
__________________________________________________________________________________________________
conv2d_8 (Conv2D)               (None, 19, 19, 42)   3066        board[0][0]                      
__________________________________________________________________________________________________
batch_normalization_8 (BatchNor (None, 19, 19, 42)   168         conv2d_8[0][0]                   
__________________________________________________________________________________________________
activation_8 (Activation)       (None, 19, 19, 42)   0           batch_normalization_8[0][0]      
____________________________________________________________________________________________

In [42]:
from tensorflow.keras.layers import Dense, Flatten, Input, Dropout, MaxPooling2D, Conv2D, BatchNormalization, Activation, AveragePooling2D, ZeroPadding2D

input = keras.Input(shape=(19, 19, planes), name='board')

x = layers.Conv2D(38, 3, padding='same')(input)
x = BatchNormalization()(x)
x = Activation('relu')(x)

x = layers.Conv2D(64, 3, padding='same')(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)

x = layers.Conv2D(128, 3, padding='same')(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = Dropout(0.3)(x)

ident = x
x = layers.Conv2D(128, 3, padding='same')(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = layers.Conv2D(128, 3, padding='same')(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = Dropout(0.3)(x)
x = layers.add([ident,x])
ident2 = x
x = layers.Conv2D(128, 3, padding='same')(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = layers.Conv2D(128, 3, padding='same')(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
#x = Dropout(0.3)(x)
x = layers.add([ident2,x])

policy_head = layers.Conv2D(2, 1, padding='same')(x)
policy_head = BatchNormalization()(policy_head)
policy_head = Activation('relu')(policy_head)
policy_head = layers.Flatten()(policy_head)
policy_head = layers.Dense(moves, activation='softmax', name='policy')(policy_head)

value_head = layers.Conv2D(32, 3, padding='same')(x)
value_head = BatchNormalization()(value_head)
value_head = Activation('relu')(value_head)
value_head = layers.Conv2D(1, 1, padding='same')(value_head)
value_head = BatchNormalization()(value_head)
value_head = Activation('relu')(value_head)
value_head = layers.ZeroPadding2D(padding=(1, 1))(value_head)
value_head = layers.AveragePooling2D(strides=2)(value_head)
value_head = layers.ZeroPadding2D(padding=(1, 1))(value_head)
value_head = layers.AveragePooling2D(strides=2)(value_head)
value_head = layers.Conv2D(1, 1, padding='same')(value_head)
value_head = layers.Flatten()(value_head)
value_head = layers.Dense(256, activation='relu')(value_head)
value_head = layers.Dense(1, activation='tanh', name='value')(value_head)

model = keras.Model(inputs=input, outputs=[policy_head, value_head])
model.summary()

#Name of the model, to save it later
now = datetime.now()
date_time = now.strftime("%m-%d-%Y_%H-%M-%S")
save_dir = os.path.join(os.getcwd(), 'models')
model_name = 'dl_proj_model_' + date_time + '.h5'

Model: "model_24"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
board (InputLayer)              [(None, 19, 19, 8)]  0                                            
__________________________________________________________________________________________________
conv2d_271 (Conv2D)             (None, 19, 19, 38)   2774        board[0][0]                      
__________________________________________________________________________________________________
batch_normalization_253 (BatchN (None, 19, 19, 38)   152         conv2d_271[0][0]                 
__________________________________________________________________________________________________
activation_253 (Activation)     (None, 19, 19, 38)   0           batch_normalization_253[0][0]    
___________________________________________________________________________________________

In [43]:
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)

model_path = os.path.join(save_dir, model_name)
#Save only the best model
#Problem : multiple output // j'ai pas encore vu comment checker les deux outputs
model_save = ModelCheckpoint(model_path, monitor='val_policy_loss', verbose=1, save_best_only=True, mode='min')
#Avoid wasting time
earlyStopping = EarlyStopping(monitor="val_policy_loss",patience=15)
callbacks_list = [model_save, earlyStopping]

model.compile(optimizer='Adam',
              loss={'value': 'mse', 'policy': 'categorical_crossentropy'})

history = model.fit(input_data, {'policy': policy, 'value': value},
           epochs=10, batch_size=64, validation_split=0.2, callbacks=callbacks_list)

print("Saved model : " + model_path)


Train on 240000 samples, validate on 60000 samples
Epoch 1/10
239872/240000 [============================>.] - ETA: 0s - loss: 4.6780 - policy_loss: 4.4372 - value_loss: 0.2408
Epoch 00001: val_policy_loss improved from inf to 3.82701, saving model to /content/gdrive/My Drive/DeepLearningProject/models/dl_proj_model_12-10-2019_22-05-45.h5
240000/240000 [==============================] - 85s 353us/sample - loss: 4.6777 - policy_loss: 4.4369 - value_loss: 0.2408 - val_loss: 4.0673 - val_policy_loss: 3.8270 - val_value_loss: 0.2406
Epoch 2/10
239872/240000 [============================>.] - ETA: 0s - loss: 3.6639 - policy_loss: 3.4242 - value_loss: 0.2397
Epoch 00002: val_policy_loss improved from 3.82701 to 3.38132, saving model to /content/gdrive/My Drive/DeepLearningProject/models/dl_proj_model_12-10-2019_22-05-45.h5
240000/240000 [==============================] - 82s 342us/sample - loss: 3.6640 - policy_loss: 3.4242 - value_loss: 0.2397 - val_loss: 3.6209 - val_policy_loss: 3.3813 - 

In [44]:
#Run and evaluate  the last
import glob
from tensorflow.keras.models import load_model

#Take the latest_file, i.e. the latest model
save_dir = os.path.join(os.getcwd(), 'models')
save_dir = save_dir+'/*'
list_of_files = glob.glob(save_dir) 
latest_file = max(list_of_files, key=os.path.getctime)
model_name = latest_file

model_path = model_name
model = load_model(model_path)
model.summary()

#score = model.evaluate(x_test,y_test,batch_size=64,verbose = 0)
#print ('Validation accuracy', model.metrics_names, score)

Model: "model_24"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
board (InputLayer)              [(None, 19, 19, 8)]  0                                            
__________________________________________________________________________________________________
conv2d_271 (Conv2D)             (None, 19, 19, 38)   2774        board[0][0]                      
__________________________________________________________________________________________________
batch_normalization_253 (BatchN (None, 19, 19, 38)   152         conv2d_271[0][0]                 
__________________________________________________________________________________________________
activation_253 (Activation)     (None, 19, 19, 38)   0           batch_normalization_253[0][0]    
___________________________________________________________________________________________

NameError: ignored